In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

## Get weights

Proposed API

```python
ResNet.resnet50(pretrained=True, dataset='imagenet')

ResNet.pretrained_models
```



In [19]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider
from pathlib import Path

provider = PretrainedWeightsProvider()
for key in provider.weights_zoo.keys():
    provider[key]

INFO:root:Loaded resnet18 pretrained weights.


INFO:root:Loaded resnet26 pretrained weights.


INFO:root:Loaded resnet34 pretrained weights.


INFO:root:Loaded mobilenet_v2 pretrained weights.


INFO:root:Loaded efficientnet_b0 pretrained weights.


...

INFO:root:Loaded vgg11_bn pretrained weights.


INFO:root:Loaded vgg13_bn pretrained weights.


INFO:root:Loaded vgg16_bn pretrained weights.


INFO:root:Loaded vgg19_bn pretrained weights.


INFO:root:Loaded densenet161 pretrained weights.


In [18]:
from glasses.nn.models import *


# model = ResNetXt.resnext101_32x8d(pretrained=True)

VGG.vgg13(pretrained=True).summary()

INFO:root:Loaded vgg13 pretrained weights.



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         Conv2dPad-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
          VGGLayer-6         [-1, 64, 112, 112]               0
         Conv2dPad-7        [-1, 128, 112, 112]          73,856
              ReLU-8        [-1, 128, 112, 112]               0
         Conv2dPad-9        [-1, 128, 112, 112]         147,584
             ReLU-10        [-1, 128, 112, 112]               0
        MaxPool2d-11          [-1, 128, 56, 56]               0
         VGGLayer-12          [-1, 128, 56, 56]               0
        Conv2dPad-13          [-1, 256, 56, 56]         295,168
             ReLU-14          [-1, 256

(tensor(133047848), tensor(133047848), tensor(507.5373), tensor(719.0525))

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
torch.load('/home/zuppif/Desktop/test.pth')

In [ ]:
from glasses.nn.blocks import Conv2dPad

x = torch.rand((1,32,9,9))

Conv2dPad(32, 64, kernel_size=9, mode = 'auto', padding=0)

In [ ]:
from glasses.nn.models import *

model =  ResNetXt.resnet18(pretrained=True)

In [ ]:
model.encoder.blocks[-1]

In [ ]:
import timm

timm.list_models('vgg')

In [ ]:
import timm
m = timm.create_model('seresnet101', pretrained=True)
m

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List
from pprint import pprint
from benchmark import benchmark
from glasses.utils.ModuleTransfer import ModuleTransfer
from functools import partial
from glasses.nn.models import *
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2
from tqdm.autonotebook import tqdm
import torchvision.transforms as T
import torch.nn as nn
import torch
from PIL import Image
from glasses.utils.PretrainedWeightsProvider import Config


models_weights = {
    'seresnet50': Config(),
#     'efficientnet_b4': Config(resize=380, input_size=380, interpolation='bicubic')
}

zoo_models_mapping = {
    'seresnet50': [partial(timm.create_model, 'seresnet101',pretrained=True), SEResNet.cse_resnet34],
   
#   'efficientnet_b4': [partial(timm.create_model, 'tf_efficientnet_b4', pretrained=True), EfficientNet.efficientnet_b4]
}



# pprint(models_weights)

In [ ]:
model

In [ ]:
model = EfficientNetPytorch.from_pretrained('efficientnet-b0')
model.set_swish(False)
summary(model.cuda(), (3, 224, 224))